In [1]:
# 필요 패키지 불러오기

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = [12, 8]
plt.rc('font', family='Malgun Gothic')

import missingno as msno

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

### 데이터 불러오기

In [2]:
# 2017 ~ 2020 년도 데이터 합치기

df = pd.read_excel('./data/업체별_온실가스사용량_탄소배출량.xlsx')

df.head()

,대상연도,할당여부,업체명,업종명,온실가스 배출량,에너지 사용량
0,2020,할당,(유)에스케이씨에보닉페록사이드코리아,산업,55013,1117
1,2020,할당,(주)HJ매그놀리아용평호텔앤리조트,건물,32772,649
2,2020,할당,(주)HM금속,산업,28355,536
3,2020,할당,(주)MSC,산업,28476,568
4,2020,할당,(주)SIMPAC,산업,408855,4761


### Null 값 제거

In [3]:
for col in df.columns:
    msg = 'column : {:>15}\t Percent of NaN value :{:.2f}%'.format(col, 100*(df[col].isnull().sum() / df[col].shape[0]))
    print(msg)

column :            대상연도	 Percent of NaN value :0.00%
column :            할당여부	 Percent of NaN value :0.00%
column :             업체명	 Percent of NaN value :0.00%
column :             업종명	 Percent of NaN value :0.00%
column :        온실가스 배출량	 Percent of NaN value :0.00%
column :         에너지 사용량	 Percent of NaN value :0.00%


In [4]:
df.shape

(4068, 6)

### 업종명 통일

- '반도체'/'디스플레이'/'전기전자' => '반도체.디스플레이.전기전자'
- '산업단지' => '산업'
- '요업'/'유리'/'유리 · 요업'' => '유리.요업'
- '음료제조업'/'식료품제조업' => '음식료품'
- '지정외 폐기물처리업 등' => '지정외 폐기물처리업'
- '하수, 폐수 및 분료 처리업' => '하수처리업'
- '발전 · 에너지' => '발전에너지'
- '전환' => '발전에너지'
- 수송 => '항공'

In [5]:
df['업종명'].replace(['반도체','디스플레이','전기전자','산업단지','요업','유리','음료제조업','식료품 제조업','지정외 폐기물처리업 등', '하수, 폐수 및 분뇨 처리업','유리 · 요업','발전 · 에너지'],
                    ['반도체.디스플레이.전기전자','반도체.디스플레이.전기전자','반도체.디스플레이.전기전자','산업','유리.요업','유리.요업','음식료품','음식료품', '지정외 폐기물처리업', '하수처리업', '유리.요업','발전에너지'],inplace=True)

In [6]:
df['업종명'].unique()

array(['산업', '건물', '폐기물', '전환', '수송', '공공기타', '교통(여객)', '교통(화물)', '건설',
       '교통(철도)', '음식료품', '담배제조업', '섬유', '기계', '석유화학', '반도체.디스플레이.전기전자',
       '자동차', '유리.요업', '철강', '비철금속', '집단에너지', '시멘트', '광업', '통신', '조선',
       '제지', '발전에너지', '교통(해운)', '지정외 폐기물처리업', '하수처리업', '항공', '목재', '정유',
       '수도'], dtype=object)

In [7]:
#df.to_excel('./업체별_온실가스사용량_탄소배출량_전처리.xlsx', index=False)

In [178]:
df

,대상연도,할당여부,업체명,업종명,온실가스 배출량,에너지 사용량
0,2020,할당,(유)에스케이씨에보닉페록사이드코리아,산업,55013,1117
1,2020,할당,(주)HJ매그놀리아용평호텔앤리조트,건물,32772,649
2,2020,할당,(주)HM금속,산업,28355,536
3,2020,할당,(주)MSC,산업,28476,568
4,2020,할당,(주)SIMPAC,산업,408855,4761
...,...,...,...,...,...,...
4063,2017,목표,주식회사 피엔알,폐기물,133595,1223
4064,2017,목표,환경시설관리 주식회사,하수처리업,48598,509
4065,2017,목표,문경시 하수도사업소,하수처리업,25387,129
4066,2017,목표,오산시 환경사업소,하수처리업,18276,246


In [8]:
df = pd.read_excel('./업체별_온실가스사용량_탄소배출량_전처리.xlsx')

In [9]:
df[]

,대상연도,할당여부,업체명,업종명,온실가스 배출량,에너지 사용량,탄소배출량
0,2020,할당,(유)에스케이씨에보닉페록사이드코리아,산업,55013,1117,56130
1,2020,할당,(주)HJ매그놀리아용평호텔앤리조트,건물,32772,649,33421
2,2020,할당,(주)HM금속,산업,28355,536,28891
3,2020,할당,(주)MSC,산업,28476,568,29044
4,2020,할당,(주)SIMPAC,산업,408855,4761,413616
...,...,...,...,...,...,...,...
4063,2017,목표,주식회사 피엔알,폐기물,133595,1223,134818
4064,2017,목표,환경시설관리 주식회사,하수처리업,48598,509,49107
4065,2017,목표,문경시 하수도사업소,하수처리업,25387,129,25516
4066,2017,목표,오산시 환경사업소,하수처리업,18276,246,18522
